# 투자 분석 AI agent

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트

In [2]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# 환경 변수 로딩

In [ ]:
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# 툴 설정

In [4]:
# 사용자 정의 재무분석 툴 로딩
from stock_analyusis import comprehensive_stock_analysis

# 웹 겸색 툴 설정
search_tool = SerperDevTool()

# 

# 에이전트 설정

## LLM 객체 생성

In [5]:
current_time = datetime.now()
llm = LLM(model = "openai/gpt-4o-mini")
invest_llm = LLM(model = "openai/o3-mini-2025-01-31")
# invest_llm = LLM(model="anthropic/claude-3-5-sonnet-20240620")

## 재무 분석가 Agent

In [6]:
# 재무 분석가
financial_analyst = Agent(
    role="Financial Analyst",
    goal="회사의 재무 상태 및 성과 분석",
    backstory="당신은 재무 제표와 비율을 해석하는 데 전문성을 갖춘 노련한 재무 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

## 시장 분석가

In [7]:
# 시장 분석가
market_analyst = Agent(
    role="Market Analyst",
    goal="회사의 시장 지위 및 업계 동향 분석",
    backstory="당신은 기업/산업 현황 및 경쟁 환경을 전문적으로 분석할 수 있는 숙련된 시장 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[search_tool],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

## 위험 분석가

In [8]:
# 위험 분석가
risk_analyst = Agent(
    role="Risk Analyst",
    goal="주식과 관련된 잠재적 위험 식별 및 평가",
    backstory="당신은 투자에서 명백한 위험과 숨겨진 위험을 모두 식별하는 예리한 안목을 갖춘 신중한 위험 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## 투자 전문가

In [9]:
# 투자 전문가
investment_advisor = Agent(
    role="Investment Advisor",
    goal="전체 분석을 기반으로 한 투자 추천 제공",
    backstory="다양한 분석을 종합하여 전략적 투자 조언을 제공하는 신뢰할 수 있는 투자 자문가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    llm=invest_llm,
    allow_delegation=False,
    verbose=True
)

In [10]:
def get_user_input():
    ticker = input("투자 자문을 구하고 싶은 기업명을 입력해주세요: ")
    return ticker

In [11]:
def create_dynamic_tasks(ticker):
    financial_analysis = Task(
        description=f"""{ticker}에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=financial_analyst,
        expected_output=f"""{ticker}의 재무 상태에 대한 종합적인 분석 보고서. 
        주요 재무 지표, 수익성, 부채 비율 등을 포함하며, 
        회사의 재무 건전성과 성과 동향에 대한 인사이트를 제공해야 합니다."""
    )

    market_analysis = Task(
        description=f"""{ticker}의 시장 위치를 분석합니다. 
        경쟁 우위, 시장 점유율, 업계 동향을 평가하세요. 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 제공하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=market_analyst,
        expected_output=f"""{ticker}의 시장 위치에 대한 상세한 분석 보고서. 
        경쟁 우위, 시장 점유율, 산업 동향을 평가하고, 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 포함해야 합니다."""
    )

    risk_assessment = Task(
        description=f"""{ticker}에 대한 투자와 관련된 주요 위험을 파악하고 평가합니다. 
        시장 위험, 운영 위험, 재무 위험 및 회사별 위험을 고려하세요. 
        종합적인 위험 프로필을 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=risk_analyst,
        expected_output=f"""{ticker} 투자와 관련된 주요 리스크에 대한 포괄적인 평가 보고서. 
        시장 리스크, 운영 리스크, 재무 리스크, 회사 특정 리스크를 고려하여 
        종합적인 리스크 분석 결과를 제시해야 합니다."""
    )

    investment_recommendation = Task(
        description=f"""{ticker}의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다. 
        주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하세요. 한글로 작성하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=investment_advisor,
        expected_output=f"""
        1. 제목 및 기본 정보
           - 회사명, 티커, 현재 주가, 목표주가, 투자의견 등
        
        2. 요약(Executive Summary)
           - 핵심 투자 포인트와 주요 재무 지표를 간단히 정리
        
        3. 기업 개요
           - 회사의 주요 사업 영역, 연혁, 시장 점유율 등
        
        4. 산업 및 시장 분석
           - 해당 기업이 속한 산업의 트렌드와 전망
        
        5. 재무 분석
           - 매출, 영업이익, 순이익 등 주요 재무지표 분석
           - 수익성, 성장성, 안정성 지표 분석
        
        6. 밸류에이션
           - P/E, P/B, ROE 등 주요 밸류에이션 지표 분석
           - 경쟁사 대비 상대 밸류에이션
        
        7. 투자 의견 및 목표주가
           - 투자의견 제시 및 근거 설명
           - 목표주가 산정 방법과 근거
        
        8. 투자 위험 요인
           - 잠재적인 리스크 요인들을 나열
        
        9. 재무제표 요약
           - 최근 몇 년간의 요약 손익계산서, 재무상태표, 현금흐름표

        """
    )

    return [financial_analysis, market_analysis, risk_assessment, investment_recommendation]

# Crew 객체 생성

In [12]:
# ticker = get_user_input()
ticker = "AAPL"
tasks = create_dynamic_tasks(ticker)

crew = Crew(
    agents=[financial_analyst, market_analyst, risk_analyst, investment_advisor],
    tasks=tasks,
    process=Process.sequential,
    output_log_file="logs/investment_advisor_run.json", 
    verbose=False
)


# crew 실행

In [13]:
result = crew.kickoff()

# Agent: Financial Analyst
## Task: AAPL에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: 2025년 06월 17일


# Agent: Financial Analyst
## Thought: Action: Updated Comprehensive Stock Analysis
## Using tool: Updated Comprehensive Stock Analysis
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
{'현재 주가': {'현재 주가': np.float64(197.14500427246094), '기준 시간': '2025-06-17 00:00:00'}, '연간 데이터': {'매출': '391,035,000,000', '매출원가': '210,352,000,000', '매출총이익': '180,683,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', '매출총이익률': '46.21%', '영업이익률': '31.51%', '순이익률': '23.97%', '매출 성장률': '2.02%', '순이익 성장률': '-3.36%', '희석주당순이익(EPS)': '$6.08', '부채비율': '84.40%'}, '분기 데이터': {'매출': '95,359,000,000', '순이익': '24,780,000,000', '매출 성장률(QoQ)': '-23.28%', '순이익 성장률(QoQ)': '-31.79%'}, '연간 재무제표 요약': {'2024-09-30': {'총수익': '391,035,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', 'EPS

In [14]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

─────────────────────────────
1. 제목 및 기본 정보
─────────────────────────────
• 회사명: Apple Inc.  
• 티커: AAPL  
• 현재 주가: 197.15 USD (2025년 6월 17일 기준)  
• 목표주가: 210.00 USD  
• 투자의견: 신중한 매수 (중립-긍정적 관점)

─────────────────────────────
2. 요약 (Executive Summary)
─────────────────────────────
Apple Inc.는 강력한 브랜드 파워와 혁신적인 기술력, 통합된 에코시스템을 바탕으로 프리미엄 시장에서 지속적인 경쟁 우위를 보유하고 있습니다. 최근 재무 지표는 높은 매출총이익률(46.21%)과 영업이익률(31.51%)을 나타내고 있으나, 순이익 감소(-3.36% 연간, -31.79% 분기)와 높은 부채비율(84.40%) 등 몇몇 리스크 요인도 존재합니다. 산업 전반에서 서비스 부문과 AI, 지속 가능성 등의 트렌드가 강해지는 가운데, AAPL은 일부 성장 둔화와 단기적 위험 필요성에도 불구하고 장기적으로는 견고한 시장 지위를 유지할 것으로 기대됩니다.

─────────────────────────────
3. 기업 개요
─────────────────────────────
Apple Inc.는 혁신적인 전자제품 및 관련 서비스를 제공하며, 대표적인 제품군(아이폰, 아이패드, 맥북, 애플 워치 등)과 디지털 생태계(iCloud, App Store, Apple Music 등)를 통해 전 세계 고객에게 높은 고객 충성도와 프리미엄 경험을 선사합니다. 오랜 역사와 축적된 기술력, 그리고 강력한 브랜드 이미지는 AAPL이 글로벌 시장에서 차별화된 경쟁력을 유지하는 주요 요인입니다. 현재 AAPL은 프리미엄 스마트폰, 태블릿, 노트북 시장에서 높은 점유율을 기록하며, 서비스 부문에서의 지속적인 확장을 통해 추가적인 매출원을 확보하고 있습니다.

─────────────────────────────
4. 산업 및 시장 분석
─────────────────────────────
• 산업 트렌드:  
  - 스마트폰 및 태블릿 시장은 기술 혁신과 사용자 경험 개선에 중점을 두고 경쟁이 치열합니다.  
  - 디지털 서비스, 클라우드 기반 서비스, AI 통합 등 차세대 기술이 산업 전반에 빠르게 확산되고 있습니다.
  
• 시장 전망:  
  - AAPL의 프리미엄 시장 내 경쟁력은 여전히 높으며, 서비스 부문과 혁신적인 제품 라인업 확대로 인해 향후 안정적 성장이 기대됩니다.
  - 그러나 경제적 불확실성, 글로벌 공급망 문제 및 경쟁사들의 기술 발전 등 외부 변수는 단기적으로 리스크로 작용할 수 있습니다.

─────────────────────────────
5. 재무 분석
─────────────────────────────
• 연간 주요 재무 지표 (2024-09-30 기준):  
  - 총 매출: 391,035,000,000 USD  
  - 영업이익: 123,216,000,000 USD  
  - 순이익: 93,736,000,000 USD  
  - EBITDA: 134,661,000,000 USD  
  - 희석주당순이익 (EPS): 6.08 USD  
  - 매출총이익률: 46.21%  
  - 영업이익률: 31.51%  
  - 순이익률: 23.97%  
  - 매출 성장률: 2.02%  
  - 순이익 성장률: -3.36%  
  - 부채비율: 84.40%

• 분기 데이터 (최근 분기: 2025년 3월 31일):  
  - 총 매출: 95,359,000,000 USD  
  - 순이익: 24,780,000,000 USD  
  - 매출 성장률(QoQ): -23.28%  
  - 순이익 성장률(QoQ): -31.79%

• 분석:  
  - AAPL은 높은 매출총이익률과 영업이익률을 유지하며 비용 관리와 판매 효율 면에서는 강점을 갖고 있음  
  - 단기적으로는 분기 실적의 감소와 순이익 마이너스 성장률이 우려되는 지표로 나타남  
  - 부채비율이 상대적으로 높아 경제 불확실성 시 재무적 리스크가 존재함

─────────────────────────────
6. 밸류에이션
─────────────────────────────
• 주요 밸류에이션 지표 (추정):  
  - P/E, P/B, ROE 등은 업계 평균과 비슷하거나 프리미엄 수준을 유지할 것으로 예상됨  
  - 프리미엄 시장에서의 강력한 브랜드 영향력으로 인해 상대적 밸류에이션은 다소 높게 형성되어 있으나 이는 장기적 성장 가능성에 대한 반영 
• 경쟁사 대비:  
  - AAPL은 경쟁사에 비해 에코시스템과 서비스 부문에서 차별화된 가치를 제공하며, 투자자들에게 안정적 수익원을 제공할 수 있는 잠재력이 있음

─────────────────────────────
7. 투자 의견 및 목표주가
─────────────────────────────
• 투자 의견:  
  - AAPL은 안정적인 이익 마진과 강한 브랜드 파워를 보유하고 있으나, 단기적인 매출·순이익 성장 둔화 및 높은 부채비율 등으로 인해 신중한 접근이 요구됩니다.  
  - 따라서 “신중한 매수” 의견을 제시하며, 포트폴리오 내 일부 비중 할당 또는 모멘텀 유지 전략을 추천합니다.
  
• 목표주가 산정 근거:  
  - 목표주가는 현재 재무 건전성과 성장 잠재력, 시장 내 경쟁 우위 및 밸류에이션 수준을 고려하여 약 210.00 USD로 산정하였습니다.  
  - 단, 단기적 리스크와 시장 변동성을 감안해 점진적 상승세를 모니터링하며 투자 결정할 필요가 있습니다.

─────────────────────────────
8. 투자 위험 요인
─────────────────────────────
• 시장 위험:  
  - 글로벌 경제 불확실성, 환율 변동 및 경쟁 심화로 인한 매출 압박  
• 운영 위험:  
  - 공급망 불안정, 기술 혁신 경쟁 및 R&D 비용 증가  
• 재무 위험:  
  - 높은 부채비율, 단기 실적 감소 및 순이익 마이너스 성장 우려  
• 규제 및 법적 위험:  
  - 반독점 문제, 앱 스토어 관련 규제 및 소비자 보호 이슈  
• 회사별 위험:  
  - 브랜드 이미지 관리 실패, 품질 및 서비스 관련 이슈 발생 시 소비자 신뢰 하락 가능성

─────────────────────────────
9. 재무제표 요약 (연간)
─────────────────────────────
• 2024-09-30:  
  - 총수익: 391,035,000,000 USD  
  - 영업이익: 123,216,000,000 USD  
  - 순이익: 93,736,000,000 USD  
  - EBITDA: 134,661,000,000 USD  
  - 희석 EPS: 6.08 USD

• 2023-09-30:  
  - 총수익: 383,285,000,000 USD  
  - 영업이익: 114,301,000,000 USD  
  - 순이익: 96,995,000,000 USD  
  - EBITDA: 125,820,000,000 USD  
  - 희석 EPS: 6.13 USD

• 2022-09-30:  
  - 총수익: 394,328,000,000 USD  
  - 영업이익: 119,437,000,000 USD  
  - 순이익: 99,803,000,000 USD  
  - EBITDA: 130,541,000,000 USD  
  - 희석 EPS: 6.11 USD

─────────────────────────────
최종 결론:
Apple Inc. (AAPL)는 여전히 강력한 브랜드와 혁신적 기술을 바탕으로 프리미엄 시장에서 선도적 위치를 유지하고 있으나, 단기적인 성장 둔화와 높은 부채비율 등 재무적 및 운영적 위험 요소로 인해 신중한 접근이 필요합니다. 단기 리스크를 감안한 포트폴리오 내 일부 비중의 할당 및 중장기 성장 모멘텀을 고려할 때, 현재 주가 197.15 USD 대비 목표주가 210.00 USD를 목표로 “신중한 매수” 전략을 추천합니다. 투자자들은 지속적으로 재무지표 및 시장 변동성을 모니터링하며, 기술 혁신과 규제 변화에 주의하여 투자 결정을 내릴 필요가 있습니다.